In [14]:
# Imports
import requests
import os

from IPython.display import HTML
from ipywidgets import interact

# Functions
def multireddit(subreddits, limit=5, orderby='score'):
    
    if subreddits == "":
        return;
    
    ''' Displays an HTML listing of the top `limit` articles from the
    various `subreddits` specified by the user.  These articles will be
    sorted by the field specified by `orderby`

      - `subreddits` is a comma-separated list of subreddits (ie.
        `linux,linuxmasterrace`)

      - `orderby` is any field in the Reddit JSON data for a particular
        article (ie. `score`, `title`, `created_utc`, etc.)
    '''

    subreddits_list = subreddits.split(",")   

    # TODO: Process function arguments
    url_list = []
    article_list = []

    for i in subreddits_list:
        new_url = 'https://www.reddit.com/r/' + i + '/.json'
        url_list.append(new_url)

    # TODO: Fetch subreddit data
    for i in url_list:
        counter = 0 

        headers  = {'user-agent': 'reddit-{}'.format(os.environ['USER'])}
        response = requests.get(i, headers=headers)

        while response.status_code != 200 and counter < 20: 
            response = requests.get(i, headers=headers)
            counter += 1 

        data = response.json()

        for child in data['data']['children']:
            article_obj = {
                'title': '',
                'score': 0,
                'url': '',
                'comments': '',
                'created_utc': 0
            }

            article_obj["title"] = child['data']['title']
            article_obj['score'] = child['data']['score']
            article_obj['url'] = child['data']['url']
            article_obj['comments'] = 'https://www.reddit.com' + child['data']['permalink']
            article_obj['created_utc'] = child['data']['created_utc']

            article_list.append(article_obj)
            
    # TODO: Sort and limit data, and construct HTML results
    if orderby == 'title' or orderby == 'url' or orderby == 'comments':
        article_list = sorted(article_list, key=lambda k: k[orderby])       
    else:
        article_list = sorted(article_list, key=lambda k: k[orderby], reverse=True)       

    html = '<table><tbody>'
    for i in range(limit):

        html += '<tr>'
        html += '<td>' + str(i+1) + '. ' + '</td>'
        html += '<td>' + '<a href="' +  article_list[i]["url"] + '">' + article_list[i]["title"] + '</a></td>'
        html += '<td>' + '. ' + '</td>'
        html += '<td>' + 'Score: ' + str(article_list[i]["score"]) + '</td>'
        html += '<td>' + '. ' + '</td>'
        html += '<td><a href="' + article_list[i]["comments"] + '">Comments</a></td>'
        html += '<td>' + '. ' + '</td>'

    html += '</tbody></table>'

    display(HTML(html))
    
ORDERBY = {
    'Score': 'score',
    'Creation Time': 'created_utc',
    'Title': 'title'
}

interact(multireddit, subreddits='', limit=(1, 20), orderby=ORDERBY)    

A Jupyter Widget

<function __main__.multireddit>